In [3]:
# ==============================================================================
# PROJECT: Kaggle Playground Series - S6E2 (Heart Disease)
# MISSION: The Super Stack (CPU Version - Safe Mode)
# ==============================================================================

import pandas as pd
import numpy as np
from google.colab import drive
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# --- STEP 1: MOUNT & LOAD ---
drive.mount('/content/drive')
TRAIN_PATH = '/content/drive/MyDrive/Nihal Data/kaggle/S6E1 - heart/train.csv'
TEST_PATH = '/content/drive/MyDrive/Nihal Data/kaggle/S6E1 - heart/test.csv'

train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

# --- STEP 2: PREPARATION ---
target_mapping = {'Absence': 0, 'Presence': 1}
y = train['Heart Disease'].map(target_mapping)
X = train.drop(['id', 'Heart Disease'], axis=1)
X_test = test.drop(['id'], axis=1)

# Categorical Encoding
for col in X.select_dtypes(include=['object']).columns:
    X[col] = X[col].astype('category').cat.codes
    X_test[col] = X_test[col].astype('category').cat.codes

# --- STEP 3: DEFINING THE TITANS (CPU SETTINGS) ---

# 1. OPTIMIZED XGBOOST (CPU Mode)
xgb_params = {
    'n_estimators': 3000,
    'learning_rate': 0.0524336,
    'max_depth': 4,
    'subsample': 0.817428,
    'colsample_bytree': 0.50969,
    'min_child_weight': 10,
    'tree_method': 'hist',  # 'hist' works great on CPU too
    'n_jobs': -1,           # Use all CPU cores
    'random_state': 42
}

# 2. OPTIMIZED LIGHTGBM (CPU Mode)
lgb_params = {
    'n_estimators': 3000,
    'learning_rate': 0.063481,
    'num_leaves': 20,
    'max_depth': 3,
    'min_child_samples': 15,
    'subsample': 0.9968,
    'colsample_bytree': 0.620,
    'reg_alpha': 0.031,
    'reg_lambda': 5.554,
    'device': 'cpu',       # <--- CHANGED TO CPU
    'verbosity': -1,
    'n_jobs': -1,          # Use all CPU cores
    'random_state': 42
}

base_models = [
    ('xgb', XGBClassifier(**xgb_params)),
    ('lgb', LGBMClassifier(**lgb_params))
]

# --- STEP 4: THE META-LEARNER ---
# n_jobs=-1 is safe here because we are on CPU now
stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression(),
    cv=5,
    stack_method='predict_proba',
    n_jobs=-1
)

# --- STEP 5: TRAINING ---
print("Training the Super Stack on CPU... (This might take ~5-8 mins)")
stack_model.fit(X, y)

# --- STEP 6: SUBMISSION ---
test_probs = stack_model.predict_proba(X_test)[:, 1]
submission = pd.DataFrame({'id': test['id'], 'Heart Disease': test_probs})
submission.to_csv('submission_v11_super_stack.csv', index=False)

print("Super Stack submission 'submission_v11_super_stack.csv' generated.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Training the Super Stack on CPU... (This might take ~5-8 mins)
Super Stack submission 'submission_v11_super_stack.csv' generated.
